This notebook implements the RNA folding QUBO of Fox et al., 2021 (RNA folding using quantum computers).

Basically, the QUBO takes on this form, where $q_i$ and $q_j$ are each potential stems:

$$H = \sum_i \{c_L(k_i-\mu)^2-c_Bk_i^2\}q_i - \sum_{i>j} (2c_Bk_ik_j\delta_{ij}+\delta^{\prime}_{ij})q_iq_j$$

Where $\delta_{ij}$ is $1$ when $i$ and $j$ are in pseudoknot and $<1$ otherwise, $\delta^{\prime}_{ij}$ is $-\infty$ when $i$ and $j$ are overlapping and $0$ otherwise, $\mu$ is the length of the longest potential stem in the set, and $c_L$ and $c_B$ are tunable constants.

In [1]:
# import packages:

import numpy as np
import pandas as pd
import math
import os
import glob

In [ ]:
def actual_stems(seq_ss, seq_ps):
    
    with open(subdirectory+"/"+seq_ss) as file:
        ss_lines = file.readlines()
    
    with open(subdirectory+"/"+seq_ps) as file:
        ps_lines = file.readlines()
    
    rna = ps_lines[1]
    
    stems_actual = []

    sip = False                       # stem in progress?
    sl = 0                            # stem length
    last_line = [0, 0, 0, 0, 0, 0]    # initiate last line

    for i in range(0, len(ss_lines)):
        line = ss_lines[i].strip().split()
        
        if (int(line[4]) != 0 and sip == False):
            sip = True
            temp = [int(line[0]), int(line[4])]
            sl += 1
        elif (int(line[4]) != 0 and sip == True and (int(last_line[4])-int(line[4]) == 1)):
            sl += 1
        elif (int(line[4]) == 0 and sip == True):
            sip = False
            temp.append(sl)
            if temp[1] > temp[0]:
                stems_actual.append(temp)
            sl = 0
        elif ((int(last_line[4])-int(line[4]) != 1) and int(last_line[4]) != 0  and sip == True):
            temp.append(sl)
            if temp[1] > temp[0]:
                stems_actual.append(temp)
            temp = [int(line[0]), int(line[4])]
            sl = 1
        
        last_line = line
        
    return stems_actual

In [ ]:
def potential_stems(seq_ps):
    
    with open(subdirectory+"/"+seq_ps) as file:
        lines = file.readlines()
    
    rna = lines[1]
    
    matrix = np.zeros((len(rna),len(rna)))
    for diag in range(0, len(matrix)):
        for row in range(0, len(matrix)-diag):
            col = row + diag
            base1 = rna[row]
            base2 = rna[col]
            if row != col:
                if ((base1 == ("A" or "a")) and (base2 == ("U" or "u"))) or ((base1 == ("U" or "u")) and (base2 == ("A" or "a"))) or ((base1 == ("G" or "g")) and (base2 == ("U" or "u"))) or ((base1 == ("U" or "u")) and (base2 == ("G" or "g"))) or ((base1 == ("G" or "g")) and (base2 == ("C" or "c"))) or ((base1 == ("C" or "c")) and (base2 == ("G" or "g"))):
                    matrix[row][col] = 1
    
    stems_potential = []
    mu = 0

    for row in range(0, len(matrix)):
        for col in range (row, len(matrix)):
            if row != col:
                if matrix[row][col] == 1:
                    temp_row = row
                    temp_col = col
                    stem = [row+1,col+1,0]
                    length = 0
                    while (matrix[temp_row][temp_col] != 0) and (temp_row != temp_col):
                        length+=1
                        temp_row+=1
                        temp_col-=1
                        if length >= 2 and col-row-2*length >= 3:
                            stem[2] = length
                            stems_potential.append(stem.copy())
                    if length > mu:
                        mu = length
    
    return [stems_potential, mu, rna, len(rna)]

In [ ]:
def potential_pseudoknots(stems_potential):

    pseudoknots_potential = []

    for i in range(len(stems_potential)):
        for j in range(i + 1, len(stems_potential)):
            
            stem1 = stems_potential[i]
            stem2 = stems_potential[j]
    
            i_a = stem1[0]
            j_a = stem1[1]
            i_b = stem2[0]
            j_b = stem2[1]
    
            pseudoknot = [i, j, 0]
    
            if (i_a < i_b and i_b < j_a and j_a < j_b) or (i_b < i_a and i_a < j_b and j_b < j_a):
        
                pseudoknot[2] = 1
    
            pseudoknots_potential.append(pseudoknot)
            
    return pseudoknots_potential

In [ ]:
def potential_overlaps(stems_potential):
    
    overlaps_potential = []
    overlap_penalty = 1e6

    for i in range(len(stems_potential)):
        for j in range(i+1, len(stems_potential)):
    
            stem1 = stems_potential[i]
            stem2 = stems_potential[j]
    
            overlap = [i, j, 0]
    
            stem1_cspan1 = set(range(stem1[1]-stem1[2]+1, stem1[1]+1))
            stem2_cspan1 = set(range(stem2[1]-stem2[2]+1, stem2[1]+1))
            
            stem1_cspan2 = set(range(stem1[0], stem1[0]+stem1[2]))
            stem2_cspan2 = set(range(stem2[0], stem2[0]+stem2[2]))
    
            if (len(stem1_cspan1 & stem2_cspan1) != 0) or (len(stem1_cspan2 & stem2_cspan2) != 0)  or (len(stem1_cspan1 & stem2_cspan2) != 0) or (len(stem1_cspan2 & stem2_cspan1) != 0):
        
                overlap[2] = overlap_penalty
        
            overlaps_potential.append(overlap)
            
    return overlaps_potential

In [ ]:
def model(stems_potential, overlaps_potential, pseudoknots_potential, cl, cb, delta):
    
    L = {}
    Q = {}
    k = 0
    
    for i in range(0, len(stems_potential[0])):
        L[str(i)] = cl*((stems_potential[0][i][2]-stems_potential[1])**2)-cb*(stems_potential[0][i][2]**2)
        for j in range(i+1, len(stems_potential[0])):
            Q[(str(i), str(j))] = -(2*cb*stems_potential[0][i][2]*stems_potential[0][j][2]*(delta*pseudoknots_potential[k][2] + (1-pseudoknots_potential[k][2]))) + overlaps_potential[k][2]
            k += 1
            
    return L, Q

In [ ]:
def energy(stems_actual, mu, cl, cb, delta):
    
    k = 0
    
    pseudoknots_actual = potential_pseudoknots(stems_actual)
    cost = 0

    for i in range(0, len(stems_actual)):
        cost += cl*((stems_actual[i][2]-mu)**2)-cb*(stems_actual[i][2]**2)
        for j in range(i+1, len(stems_actual)):
            cost += -2*cb*stems_actual[i][2]*stems_actual[j][2]*(delta*pseudoknots_actual[k][2] + (1-pseudoknots_actual[k][2]))
            k += 1
    
    return cost

In [ ]:
# function to compare actual and predicted structure based on comparison of base-pairs:

def evaluation_1(stems_actual, stems_potential):
    
    bp_actual = []
    bp_predicted = []

    for i in range(0, len(stems_actual)):
        for j in range(0, stems_actual[i][2]):
            bp_actual.append((stems_actual[i][0]+j, stems_actual[i][1]-j))
        
    for i in range(0, len(stems_potential)):
        for j in range(0, stems_potential[i][2]):
            bp_predicted.append((stems_potential[i][0]+j, stems_potential[i][1]-j))
            
    TP = 0    # number of correctly identified base pairs
    FP = 0    # number of the predicted base pairs missing from the known structure
    FN = 0    # number of non-predicted base pairs present in the known structure

    for i in range(0, len(bp_predicted)):
        if bp_predicted[i] in bp_actual:
            TP += 1
        else:
            FP += 1

    for i in range(0, len(bp_actual)):
        if bp_actual[i] not in bp_predicted:
            FN += 1
            
    if TP+FP != 0:
        ppv = TP/(TP+FP)
    else:
        ppv = 0
    if TP+FN != 0:
        sensitivity = TP/(TP+FN)
    else:
        sensitivity = 0
    
    return [ppv, sensitivity]

In [ ]:
# function to compare actual and predicted structure based on comparison of bases involved in pairing:

def evaluation_2(stems_actual, stems_predicted):
    
    pb_actual = []
    pb_predicted = []

    for i in range(0, len(stems_actual)):
        for j in range(0, stems_actual[i][2]):
            pb_actual.append(stems_actual[i][0]+j)
            pb_actual.append(stems_actual[i][1]-j)
        
    for i in range(0, len(stems_predicted)):
        for j in range(0, stems_predicted[i][2]):
            pb_predicted.append(stems_predicted[i][0]+j)
            pb_predicted.append(stems_predicted[i][1]-j)
            
    TP = 0    # number of correctly identified bases that are paired
    FP = 0    # number of the predicted paired bases missing from the known structure
    FN = 0    # number of non-predicted paired bases present in the known structure

    for i in range(0, len(pb_predicted)):
        if pb_predicted[i] in pb_actual:
            TP += 1
        else:
            FP += 1

    for i in range(0, len(pb_actual)):
        if pb_actual[i] not in pb_predicted:
            FN += 1
            
    if TP+FP != 0:
        ppv = TP/(TP+FP)
    else:
        ppv = 0
    if TP+FN != 0:
        sensitivity = TP/(TP+FN)
    else:
        sensitivity = 0
    
    return [ppv, sensitivity]

In [16]:
# function to compare actual and predicted structure based on comparison of base-pairs:

def MCC(stems_actual, stems_predicted, stems_potential):
    
    bp_actual = []
    bp_predicted = []
    bp_potential = []

    for i in range(0, len(stems_actual)):
        for j in range(0, stems_actual[i][2]):
            bp_actual.append((stems_actual[i][0]+j, stems_actual[i][1]-j))
        
    for i in range(0, len(stems_predicted)):
        for j in range(0, stems_predicted[i][2]):
            bp_predicted.append((stems_predicted[i][0]+j, stems_predicted[i][1]-j))
            
    for i in range(0, len(stems_potential)):
        for j in range(0, stems_potential[i][2]):
            bp_potential.append((stems_potential[i][0]+j, stems_potential[i][1]-j))
            
    TP = 0    # number of correctly identified base pairs
    TN = 0    # number of correctly excluded base pairs
    FP = 0    # number of the predicted base pairs missing from the known structure
    FN = 0    # number of non-predicted base pairs present in the known structure

    for i in range(0, len(bp_predicted)):
        if bp_predicted[i] in bp_actual:
            TP += 1
        else:
            FP += 1

    for i in range(0, len(bp_actual)):
        if bp_actual[i] not in bp_predicted:
            FN += 1
            
    for i in range(0, len(bp_potential)):
        if (bp_potential[i] not in bp_predicted) and (bp_potential[i] not in bp_actual):
            TN += 1
    
    if TP + FP != 0 and TP + FN != 0 and TN + FP != 0 and TN + FN != 0:
        score = (TP*TN-FP*FN)/np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    else:
        score = -1e6
        
    return score

In [ ]:
def connectivity_table(bpRNA_id, sequence, stems, t):
    print(len(sequence), bpRNA_id, file=open("./results/cts/"+bpRNA_id+"_"+t+"_model1.ct", "w"))
    for i in range(0, len(sequence)):
        pair = 0
        for j in stems:
            for k in range(j[0], j[0]+j[2]):
                if i+1 == k:
                    pair = j[1]+j[0]-k
            for k in range(j[1]-j[2]+1, j[1]+1):
                if i+1 == k:
                    pair = j[0]+j[1]-k
        print(i+1, sequence[i], i, i+2, pair, i+1, file=open("./results/cts/"+bpRNA_id+"_"+t+"_model1.ct", "a"))

In [ ]:
# connecting with D-Wave:

from dwave.cloud import Client

client = Client.from_config(token="DEV-6b38e4697eaa586b361595c629788f595b810a14")
client.get_solvers()

from dwave.system.samplers import DWaveSampler
from dwave.system.samplers import LeapHybridSampler
from dwave.system.composites import EmbeddingComposite

import dimod

sampler_q = EmbeddingComposite(DWaveSampler(token="DEV-6b38e4697eaa586b361595c629788f595b810a14", solver={'topology__type': 'pegasus'}))
sampler_h = LeapHybridSampler(token="DEV-6b38e4697eaa586b361595c629788f595b810a14")

The following cell runs all structures of a given folder (`subdirectory`), where:

- `cl` is to be tuned
- `cb` is to be tuned
- `delta` is the pseudoknot penalty, and is to be tuned

1. First, the actual stems are found, and then their energies. 
2. Second, the potential stems are found, as well as their potential overlaps and pseudoknots. 
3. Third, the QUBO model for is built. 
4. Fourth, the model is run on DWave.
5. Fifth, connectivity table files are built for the actual and predicted structures.
6. Sixth, the predicted structure is evaluated against the actual structure using BP and PB Sensitivity and PPV, as well as the Mathews Correlation Coefficient.

We will need to take this cell and modify it for both the training and testing steps to our protocol.

In [ ]:
cl = 1
cb = 1
delta = 1

subdirectory = "./data/woutPKs/s"

ct = [f for f in os.listdir(subdirectory) if f.endswith('.ct.txt')]
fasta = [f for f in os.listdir(subdirectory) if f.endswith('.fasta.txt')]

bprna_id       = [] # IDs

a_stems        = [] # actual structure stems
a_energies     = [] # actual structure energies

p_stems        = [] # potential structure stems
p_overlaps     = [] # potential structure overlaps
p_pseudoknots  = [] # potential structure pseudoknots

models         = [] # models

problem = []       # intiate list of problems
prediction = []    # intiate list of predictions
evaluation = []    # initiate list of evaluations
min_time = []      # intiate list of time to solution

for i in range(0, len(ct)):

    try:                                                            # try/except here b/c some wrong CT files
        print("building model for:", ct[i].split('.')[0])
        bprna_id.append(ct[i].split('.')[0])                        # append ID of structure
        
        p_stems.append(potential_stems(fasta[i]))                   # find potential stems of structure
        p_overlaps.append(potential_overlaps(p_stems[i][0]))        # find potential overlaps
        p_pseudoknots.append(potential_pseudoknots(p_stems[i][0]))  # find potential pseudoknots
        
        a_stems.append(actual_stems(ct[i], fasta[i]))               # find actual stems of structure
        a_energies.append(energy(a_stems[i], p_stems[i][1], cl, cb, delta))        # compute energy of actual structure
        
        models.append(model(p_stems[i], p_overlaps[i], p_pseudoknots[i], cl, cb, delta))
    
    except:
        print("error in preprocessing, skipping...")
    
    try:
        print("running model for:", ct[i].split('.')[0])
        problem.append(dimod.BinaryQuadraticModel(models[i][0], models[i][1], vartype = 'BINARY', offset = 0.0))    

        #sampleset = sampler_q.sample(problem[i], num_reads=1000)
        #min_time.append("placeholder")
        sampleset = sampler_h.sample(problem[i])              # hybrid
        min_time.append(sampler_h.min_time_limit(problem[i])) # hybrid
        
        for datum in sampleset.data(['sample', 'energy', 'num_occurrences']):
            results = datum.sample
            predicted_energy = datum.energy
        
        stems_found = []           # initiate list of predicted stems

        for j in range(0, len(results)):
            if results[str(j)] == 1:
                stems_found.append(p_stems[i][0][j])
            
        prediction.append([stems_found, predicted_energy]) # record predicted stems and structure energy
                
        connectivity_table(bprna_id[i], p_stems[i][2], stems_found, "predicted") # write predicted CT file
        connectivity_table(bprna_id[i], p_stems[i][2], a_stems[i], "actual")     # write actual CT file
        
    except:
        print("no embedding found, skipping...")

    try:
        print("evaluating model for:", ct[i].split('.')[0])
        metrics_1 = []
        metrics_2 = []
        mcc = []
        metrics_1.append(evaluation_1(a_stems[i], stems_found))                  # compute BP metrics
        metrics_2.append(evaluation_2(a_stems[i], stems_found))                  # compute PB metrics
        mcc.append(MCC(a_stems[i], stems_found, p_stems[i][0])) 
        evaluation.append((metrics_1, metrics_2, mcc))
    
    except:
        print("no structure found, skipping...")